In [13]:
import subprocess
def install(module=""):
    if not module:
        module = input("Module: ")
    result = subprocess.run(['venv/Scripts/pip','install', module], capture_output=True, text=True)
    print("------------------------------------------------------------------------------")
    print(result.stdout if result.stdout else "")
    print(result.stderr if result.stderr else "")
    print("------------------------------------------------------------------------------")

In [2]:
install()

Module:  selenium


------------------------------------------------------------------------------
  Using cached selenium-4.8.0-py3-none-any.whl (6.3 MB)
  Using cached urllib3-1.26.14-py2.py3-none-any.whl (140 kB)
  Using cached trio-0.22.0-py3-none-any.whl (384 kB)
  Using cached trio_websocket-0.9.2-py3-none-any.whl (16 kB)
  Using cached certifi-2022.12.7-py3-none-any.whl (155 kB)
  Using cached attrs-22.2.0-py3-none-any.whl (60 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
  Using cached async_generator-1.10-py3-none-any.whl (18 kB)
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Using cached outcome-1.2.0-py2.py3-none-any.whl (9.7 kB)
  Using cached sniffio-1.3.0-py3-none-any.whl (10 kB)
  Using cached cffi-1.15.1-cp311-cp311-win_amd64.whl (179 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
  Using cached pycparser-2.21-py2.py3-none-any.whl (118 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)


[noti

In [3]:
install("selenium")
install("chromedriver-binary")
install("pandas")
install("bs4")

------------------------------------------------------------------------------


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: D:\PythonProjects\NoteProjects\BotProject\venv\Scripts\python.exe -m pip install --upgrade pip

------------------------------------------------------------------------------
------------------------------------------------------------------------------
  Using cached chromedriver-binary-110.0.5481.30.0.tar.gz (5.1 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for chromedriver-binary: started
  Running setup.py install for chromedriver-binary: finished with status 'done'

  DEPRECATION: chromedriver-binary is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--us

In [24]:
from selenium import webdriver as wd
import chromedriver_binary
import random
import time
import os
import re
import pandas as pd
from IPython.display import clear_output
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException

In [25]:
class Element:
    def __init__(self, xpath, action="-click-", wait=False, sleep=0.0):
        self.xpath = xpath
        self.action = action
        self.wait = wait
        self.sleep = sleep

In [26]:
l_elm = []
page = ""

In [27]:
t_s = 0.5
def do_element(d, xpath, action="-click-", wait=False, sleep=0):
    global t_s
    if d and d.service.is_connectable():
        try:
            el = WebDriverWait(d, 10).until(EC.presence_of_element_located((By.XPATH, xpath))) if wait else \
            d.find_element(by=By.XPATH, value=xpath)
        except:
            print("Not found.")
            return
        if action == "-click-":
            el.click()
        elif action == "-none-":
            print("Found element.")
        elif action == "-wait-":
            input("Waiting Enter.")
        else:
            el.send_keys(action)
        if sleep:
            time.sleep(sleep)
    else:
        print("Driver not found.")

In [18]:
def file_elements(namefile, load=False):
    global l_elm, page
    
    if load:
        with open (f"{namefile}.txt", "r") as f:
            l_elm = []
            data = f.read().splitlines()
            for d in data:
                p = re.findall(r'\{\{(.*)\}\}', d)
                page = p[0] if p else page
                elm = d.split(";;")
                if len(elm) == 4:
                    l_elm.append(Element(elm[0], elm[1], elm[2], float(elm[3])))
    else:
        with open(f'{namefile}.txt', 'w') as f:
            f.write(f'{{{{{page}}}}}\n')
            for elm in l_elm:
                f.write(f'{elm.xpath};;{elm.action};;{elm.wait};;{elm.sleep}\n')
    print("Done.")

In [19]:
def add_elm():
    global l_elm
    
    xp = input("XPath: ")
    ac = input("Action (Click): ")
    ac = "-click-" if ac == "" else ac
    wa = False if input("Wait (y/N) : ") not in ["y", "Y"] else True
    sl = input("Time Sleep (default 0) : ")
    sl = 0 if sl not in "123456789" or not sl else float(sl)
      
    if xp:
        l_elm.append(Element(xp, ac, wa, sl))
        print("Done.")
    else:
        print("Error: Xpath not defined.")

In [20]:
def init_driver(headless=False):
    global page
    options = wd.ChromeOptions()
    if headless:
        options.add_argument("--headless")
    d = wd.Chrome(options=options)
    d.implicitly_wait(5)
    if page:
        d.get(page)
    return d

In [21]:
def driver_calive(d, sil=False, act=True):
    if not d or not d.service.is_connectable():
        return init_driver(sil) if act else False
    try:
        d.title
        return d if act else True
    except WebDriverException:
        return init_driver(sil) if act else False
    

In [ ]:
print("Bot Recorder")
d = init_driver()
sil = False
while True:
    print("--------------")
    print("1.- Add")
    print("2.- Remove")
    print("3.- List")
    print("4.- Test")
    print("5.- Sequence")
    print("6.- Import")
    print("7.- Export")
    print("8.- Toggle Silent")
    print("9.- Close")
    print("--------------")
    inp = input("Order >")
    clear_output()
    print("--------------")
    
    if inp not in "123456789":
        print("Choose 1-9.")
        continue
    else:
        inp = int(inp)
    
    d = driver_calive(d, sil)
        
    if inp == 1:
        add_elm()
        if l_elm:
            inp = input("Test? (y/N): ")
            if inp in ["y", "Y"]:
                do_element(d, l_elm[-1].xpath, '-none-', l_elm[-1].wait, l_elm[-1].sleep)
    elif inp == 2:
        print("Page: ", page)
        for i, elm in enumerate(l_elm):
            print(f"{i+1}.- {elm.xpath} {elm.action} {elm.wait} {elm.sleep}")
        inp = input("Enter Number (c to cancel)(page to page)")
        if inp == "c":
            continue
        if inp == "page":
            page = ""
            continue
        inp = len(l_elm)-1 if inp == "" or inp == "0" or inp not in "123456789" else int(inp)-1
        l_elm.pop(inp)
    elif inp == 3:
        print("Page: ", page)
        if l_elm:
            for i, elm in enumerate(l_elm):
                print(f"{i+1}.- {elm.xpath} {elm.action} {elm.wait} {elm.sleep}")
    elif inp == 4:
        if l_elm:
            print("Page: ", page)
            for i, elm in enumerate(l_elm):
                print(f"{i+1}.- {elm.xpath} {elm.action} {elm.wait} {elm.sleep}")
            inp = input("Enter Number (c to cancel)")
            if inp == "c":
                continue
            inp = len(l_elm)-1 if inp == "" or inp == "0" or inp not in "123456789" else int(inp)-1
            inp_1 = input("Do Action? (Y/n): ")
            inp_1 = "-none-" if inp_1 in ["N", "n"] else l_elm[inp].action
            inp_2 = input("Use Page? (y/N): ")
            if inp_2 in ["Y", "y"]:
                d = init_driver(sil)
            do_element(d, l_elm[inp].xpath, inp_1, l_elm[inp].wait, l_elm[inp].sleep)
                           
        else:
            print("List not found")
            add_elm()
            do_element(d, l_elm[-1].xpath, l_elm[-1].action, l_elm[-1].wait, l_elm[-1].sleep)
    
    elif inp == 5:
        inp = input("Press Enter to sequence each element added. (c to cancel)(page for not page)")
        if inp == "c":
            continue
        if inp != "page":
            d = init_driver(sil)
        for elm in l_elm:
            do_element(d, elm.xpath, elm.action, elm.wait, elm.sleep)
        print("Done.")
    
    elif inp == 6:
        inp = input("Namefile (.txt)(empty to cancel): ")
        if not inp:
            continue
        if os.path.exists(f"{inp}.txt"):
            file_elements(inp, True)
            print("Imported.")
        else:
            print(f"File {inp}.txt doesn't exist.")
    
    elif inp == 7:
        inp = input("Namefile (.txt)(empty to cancel): ")
        if not inp:
            continue
        file_elements(inp)
        print("Exported.")
        
    elif inp == 8:
        print("Done.")
        sil = not sil
        d = init_driver(sil)
    
    elif inp == 9:
        print("Closed.")
        break
d.close()

--------------


Press Enter to sequence each element added. (c to cancel)(page for not page) 


Not found.
Not found.
Not found.
Not found.
Not found.
Not found.
Not found.
Not found.
Done.
--------------
1.- Add
2.- Remove
3.- List
4.- Test
5.- Sequence
6.- Import
7.- Export
8.- Toggle Silent
9.- Close
--------------


In [51]:
wd = wd.Chrome()
wd.implicitly_wait(10)

In [71]:
wd.get("https://cloud.google.com/")

In [72]:
el = WebDriverWait(wd, 10).until(
      # Please Change These Down Below
        EC.presence_of_element_located((By.XPATH, '//*[@id="devsite-user"]/div/a'))
    )
el.click()

In [73]:
el = WebDriverWait(wd, 10).until(
      # Please Change These Down Below
        EC.presence_of_element_located((By.XPATH, '//*[@id="identifierId"]'))
    )
el.send_keys("pyrosvchl5")
time.sleep(t_s)

In [74]:
el = wd.find_element(by=By.XPATH, value='//*[@id="identifierNext"]/div/button')
el.click()

In [75]:
el = WebDriverWait(wd, 10).until(
      # Please Change These Down Below
        EC.presence_of_element_located((By.XPATH, '//*[@id="password"]/div[1]/div/div[1]/input'))
    )
el.send_keys("pyro2000")
time.sleep(t_s)

In [76]:
el = wd.find_element(by=By.XPATH, value='//*[@id="passwordNext"]/div/button')
el.click()

In [77]:
el = WebDriverWait(wd, 10).until(
      # Please Change These Down Below
        EC.presence_of_element_located((By.XPATH, '//*[@id="cloud-site"]/div[2]/section[1]/div/div[1]/div/div/div/cloudx-free-trial-eligible-content/slot/a[1]'))
    )
el.click()
time.sleep(t_s)

In [102]:
wd.switch_to.window(wd.window_handles[-1])

In [103]:
el = WebDriverWait(wd, 10).until(
      # Please Change These Down Below
        EC.presence_of_element_located((By.XPATH, '//*[@id="_0rif_mat-form-field-label-3"]'))
    )
el.click()
time.sleep(t_s)

In [104]:
el = WebDriverWait(wd, 10).until(
      # Please Change These Down Below
        EC.presence_of_element_located((By.XPATH, '//*[@id="_0rif_mat-option-226"]'))
    )
el.click()
time.sleep(t_s)

In [105]:
el = wd.find_element(by=By.XPATH, value='//*[@id="_0rif_mat-checkbox-1"]/label/span[1]')
el.click()
time.sleep(t_s)

In [106]:
el = wd.find_element(by=By.XPATH, value='//*[@id="main"]/div/div/central-page-area/div/div/pcc-content-viewport/div/div/pangolin-home/cfc-router-outlet/div/ng-component/signup-page/div/div/div[2]/div/div[1]/div[2]/div/cfc-router-outlet/div/cfc-free-trial-tos-form/form/signup-form-button-row/div/div/cfc-progress-button/div[1]/button')
el.click()

In [113]:
el = WebDriverWait(wd, 10).until(
      # Please Change These Down Below
        EC.presence_of_element_located((By.XPATH, '//*[@id="main"]/div/div/central-page-area/div/div/pcc-content-viewport/div/div/pangolin-home/cfc-router-outlet/div/ng-component/signup-page/div/div/div[2]/div/div[1]/div[2]/div/cfc-router-outlet/div/cfc-free-trial-billing-form/form/signup-form-button-row/div/div/cfc-progress-button/div[1]/button'))
    )
print("Done, now to next step.")

Done, now to next step.


In [5]:
soup = BeautifulSoup(wd.page_source,features="html.parser")

items = soup.find("input", {"class": "gLFyf gsfi"})

print(items)

In [10]:
soup = BeautifulSoup(wd.page_source,features="html.parser")
items = soup.find("div", {"class": "c8d6zd xWMiCc REww7c"})

print(items.text)